In [24]:
import os, sys
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
from sklearn.metrics import *
from sklearn.model_selection import *
import random
from random import shuffle
import re
import string
import unicodedata as udata
import pickle
from tqdm import tqdm # pip install tqdm

## Parameters have to be changed

最後結果輸出於all_features變數中

shape=(筆數,pad_length,vector)

修改好以下路徑就可以直接run all了

training大概要處理半小時，testing大約三分鐘



In [25]:
word_model_dict = pickle.load(open('model/word_model_dict.pkl','rb')) # word2vec pickle
df = pd.read_csv('data/train_label_10.csv',header=None) ##traing label path
df2 = pd.read_csv('data/test_label_10.csv',header=None) ## testing label path
trace_dir = 'data/trace_by_family_5up/' #trace.hooklog所在的目錄，此目錄底下還會包含各家族的子目錄


In [26]:
pad_length = 555 # 一個malware要取幾個api call sequence
train_or_test = 'train' #處理訓練料或是測試資料
vector=67 #一個api call要用幾個維度來表示。不加api預設67，加的話99。基本上不用動

## Functions

In [27]:
def tick_mapping(x):
    if  x.find('#')!=-1:
        return '#'
    else:
        return x

In [28]:
def split(x):
    if x.find('='):
        return x.lower().split('=')[-1]
    else:
        return x.lower()
        

In [29]:
def find_api(x):
#     if ['=','\\','/',':',';','-','<','.' ] not in x:
    if  x.find('=')==-1 and x.find('#')==-1 and x.find('/')==-1:
        x = re.sub('[^a-zA-Z]+','',x)
        return x
    else:
        return np.nan

In [30]:
def find_param(x):
    if  x.find('=')!=-1 and x.find('#')==-1 :
        return x.replace('=',' ').lower()
    else:
        return np.nan

In [31]:
def find_param_r(x):
    if  x.find('=')!=-1 and x.find('#')==-1 :
#         x = re.sub(r"([A-Z])",r" \1",x)
        x = re.sub(r"([A-Z][a-z])",r" \1",x)
        x = re.sub(r"([A-Z]+[ ])",r' \1',x) # 把大寫依照字拆開
        x = x.split('=')[-1].lower()
#         x = re.sub('_+|/+|-+|\.+|{+|}+|[+|]+|(+|)+|\\+|~+|^+|@+|#+|!+|%+|&+|\*+|\++|=+|?+',' ',x)
        x = re.sub("[^A-Za-z0-9]+",' ',x)
        return x
    else:
        return np.nan

In [32]:
def find_param_r_2(x):
    x = re.sub(r"([A-Z][a-z])",r" \1",x)
    x = re.sub(r"([A-Z]+[ ])",r' \1',x) # 把大寫依照字拆開
    x = x.split('=')[-1].lower()
    #         x = re.sub('_+|/+|-+|\.+|{+|}+|[+|]+|(+|)+|\\+|~+|^+|@+|#+|!+|%+|&+|\*+|\++|=+|?+',' ',x)
    x = re.sub("[^A-Za-z0-9]+",' ',x)
    return x

In [33]:
def find_param_l(x):
    if  x.find('=')!=-1 and x.find('#')==-1 :
        return x.split('=')[0].lower().strip()
    else:
        return np.nan

In [34]:
def transform_malware_to_dict(s):
    """
    Transfer a malware to a list of dict.
    Input:
    - s: pandas.series, malware
    
    Output: a list of malware attributes
    """
    list_ = []
    temp_dict = {}
    for k in s:
        if k == '#':
            if bool(temp_dict):
                list_.append(temp_dict)
            temp_dict = {}
        elif k.find('=') == -1:
            temp_dict['api'] = k
        else:
            sp = k.split('=')
            temp_dict[sp[0]] = sp[1] if sp[1] else ''
#     print(list_)
    return list_
def transform_api_to_feature(d):
    """
    Transform a dict of api to trainable features.
    Input:
    - d: dict, api keys and values
    - word_model_dict: a dict of trained doc2vec model
    - dim_dict: a dict of doc2vec dimention
    
    Output: tranable features np.array
    """
    hex2dec_list = ['lpcbData', 'lpdwDisposition', 'phkresult', 'samDesired']
    special_list = ['Return', 'uExitCode', 'lpData', 'lpType']
    word_vector_list = ['dwDesiredAccess', 'dwShareMode', 'hkey', 'hObject', 'lpFileName', 
                        'lpSubKey', 'lpValueName', 'procName', 'dwCreationDisposition', 'dwType']
    dec_list = ['dwFlags', 'dwFlagsAndAtrributes', 'dwIndex']
    
    dim_dict = {
        'dwDesiredAccess':10,
        'dwShareMode':3,
        'hkey':15,
        'hObject':4,
        'lpFileName':5,
        'lpSubKey':4,
        'lpValueName':3,
        'procName':2, 
        'dwCreationDisposition':3, 
        'dwType':2
    }
    
    output_features = []
    
    for key in hex2dec_list:
        if key in d.keys():
            f = int(d[key], 16)
        else:
            f = -1
        output_features.append([f])
        
    for key in word_vector_list:
        if key in d.keys():
            f = word_model_dict[key.lower()].infer_vector([d[key]]).tolist()
        else:
            f = [-1] * dim_dict[key]
        output_features.append(f)
    
    for key in dec_list:
        if key in d.keys():
            try:
                f = float(d[key])
            except:
                f = int(d[key], 16)
        else: 
            f = -1
        output_features.append([f])
        
    key = 'Return'
    if key in d.keys():
        if d[key].lower() == 'success':
            f = 1
        elif d[key].lower() == 'failure':
            f = 0
        else:
            f = -1
    else:
        f = -1
    output_features.append([f])
    
    key = 'uExitCode'
    if key in d.keys():
        if int(d[key]) > 255/2:
            f = 1
        else:
            f = 0
    else:
        f = -1
    output_features.append([f])
    
    key = 'lpData'
    if key in d.keys():
        f = []
        f.append(len(d[key]))
        f.append(float(d[key].replace('.', '', 1).isdigit()))
    else:
        f = [-1, -1]
    output_features.append(f)
    
    key = 'lpType'
    if key in d.keys():
        f = [0] * 5
        label = ['0', 'REG_DWORD', 'REG_BINARY', 'REG_SZ', 'REG_EXPAND_SZ']
        for i in range(len(label)):
            if d[key] == label[i]:
                f[i] = 1
    else:
        f = [-1] * 5
    output_features.append(f)
    
#     key='api'
#     if key in d.keys():
#         f = [0] * 32
#         label = ['WinHttpWriteData','GetUrlCacheEntryInfo', 'TerminateProcess','OpenThread','RegDeleteKey',
# 'HttpSendRequest','RegOpenKey','LoadLibrary','RegQueryValue','RegCloseKey', 'RegCreateKey',
#  'CreateFile', 'CloseHandle', 'RegOpenCurrentUser', 'RegEnumValue', 'CreateRemoteThread',
#  'CreateThread',  'DeleteFile', 'RegSetValue', 'ExitProcess', 'OpenProcess', 'CopyFile',
#  'RegDeleteValue', 'CreateProcess', 'WinHttpOpen', 'WinHttpConnect', 'WinHttpOpenRequest',
#  'WinHttpSendRequest',  'WinHttpReadData', 'InternetOpen', 'InternetConnect', 'WinExec']
#         for i in range(len(label)):
#             if d[key] == label[i]:
#                 f[i] = 1
#     else:
#         f = [-1] * 32
#     output_features.append(f)
#     print(np.array(output_features))
    return np.concatenate(output_features)

In [35]:
def clean_values(d):
    a={}
    for key, value in d.items():
        if key == 'api':
            a[key] =  re.sub('[^a-zA-Z]+','',value)
#             continue
        else:
            d[key] = find_param_r_2(value)
    return d

## Main

In [36]:
df.drop([0],inplace=True)
df2.drop([0],inplace=True)
train_list = df[0].values.tolist()
test_list = df2[0].values.tolist()

In [37]:
pad_len = pad_length
if train_or_test == 'train':
    train_test = train_list
else:
    train_test = test_list
df_all = pd.DataFrame( columns=['api'])
df2 = pd.DataFrame( columns=['param_left','param_right'])
df_all2 = pd.DataFrame( columns=['param_left','param_right'])
all_feature = np.empty((0,vector)) #99
all_features = np.empty((0,pad_len,vector)) #99
x = np.array([0]*vector) #99
kk = pd.Series([x])
for name in tqdm(os.listdir(trace_dir)):
    for hl in os.listdir(trace_dir+name):
#         print(name,hl) #debug用
        if hl.split('.')[0] not in train_test:  #train_list/test_list
            continue
        try:
            df = pd.read_csv(trace_dir+name+"/"+hl ,header=None,encoding='latin1',sep='\n')
        except:
            df = pd.read_table(trace_dir+name+"/"+hl ,header=None,encoding='latin1')
        df = df.drop(df.iloc[0].index,axis=0)
        df[0] = df[0].map(tick_mapping)
#         print(len(df[0]))
        malware_dict_list = transform_malware_to_dict(df[0])
#         print(len(malware_dict_list))
        malware_dict_list = pd.Series(data=malware_dict_list).map(clean_values)
#         api_call = api_call.map(encode_api)
        malware_dict_list = malware_dict_list.map(transform_api_to_feature)
        api_length = len(malware_dict_list)
        if api_length <pad_len:  #pad size
            for i in range(pad_len-api_length):
                malware_dict_list = malware_dict_list.append(kk)
#                 print(api_length)
        else:
            malware_dict_list = malware_dict_list[:pad_len]
#         break
#         print(len(malware_dict_list))

        features = malware_dict_list.values#.tolist()
#         print(features.shape,feature_a.shape)
        all_feature = np.append(all_feature, features)
#         print(all_feature[0].shape)
        all_feature = np.stack(all_feature,axis=0)
        all_feature = np.expand_dims(all_feature,axis=0)
#         print(all_feature.shape, all_features.shape)
        all_features = np.append(all_features,all_feature,axis=0)
        all_feature = np.empty((0,vector)) #99
# df_all2.head()
all_features.shape

100%|██████████| 52/52 [01:25<00:00,  1.64s/it]


(422, 555, 67)